## Evaluating models in terms of face generation quality



This code replicates the results from the paper https://arxiv.org/abs/2210.00586 using local device storage
downloading and extracting datasets in https://drive.google.com/file/d/16BXO1fgN08UGLLeA5ZNU9bhwAkcAOdci/view is necessary

Github repository https://github.com/aliborji/GFW

Install necessary dependencies

In [2]:
!pip install torch torchvision numpy scipy pandas pillow tqdm



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# ====================================
# STEP 1: IMPORT LIBRARIES
# ====================================
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from scipy import linalg
import torch.nn as nn

Replace these paths with your local paths where the datasets are stored

The datasets used in the original research paper can be found in https://drive.google.com/file/d/16BXO1fgN08UGLLeA5ZNU9bhwAkcAOdci/view
Download, extract them, and save in different folders
the datasets used in the analysis are the "faces" datasets


In [4]:
# ====================================
# STEP 2: DEFINE PATHS (LOCAL SETUP)
# ====================================
real_path = r'C:\Users\hugo_\Documents\hugo_garcia\Big_Data_Analytics_georgian_college\AI management\1 semester\Machine Learning Programming\Final Project\datasets\real\real_faces'
sd_path = r'C:\Users\hugo_\Documents\hugo_garcia\Big_Data_Analytics_georgian_college\AI management\1 semester\Machine Learning Programming\Final Project\datasets\stable_diffusion\generated_data\faces_generated'
mj_path = r'C:\Users\hugo_\Documents\hugo_garcia\Big_Data_Analytics_georgian_college\AI management\1 semester\Machine Learning Programming\Final Project\datasets\midjourney\faces_generated_midjourney'
dalle_path = r'C:\Users\hugo_\Documents\hugo_garcia\Big_Data_Analytics_georgian_college\AI management\1 semester\Machine Learning Programming\Final Project\datasets\dalle2\DALLEFaces'


Load the images and convert convert them into the same format

In [11]:
# ====================================
# STEP 3: IMAGE LOADING FUNCTION
# ====================================
def get_activations(folder, model, batch_size=50, max_images=2000):
    model.eval()
    preprocess = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3),
    ])

    images = []
    image_files = [f for f in os.listdir(folder) if f.lower().endswith(('jpg', 'png', 'jpeg'))]
    image_files = image_files[:max_images]  # limit number of images

    for file in tqdm(image_files, desc=f'Loading images from {os.path.basename(folder)}'):
        img_path = os.path.join(folder, file)
        img = Image.open(img_path).convert('RGB')
        img = preprocess(img)
        images.append(img)

    images = torch.stack(images)
    activations = []

    with torch.no_grad():
        for i in range(0, len(images), batch_size):
            batch = images[i:i+batch_size]
            output = model(batch)
            activations.append(output.cpu().numpy())

    activations = np.concatenate(activations, axis=0)
    return activations


InceptionV3 model is used to score each model according to the similarity between their generated images and real ones (FID score)

In [12]:
# ====================================
# STEP 4: LOAD INCEPTIONV3 MODEL
# ====================================
from torchvision.models import inception_v3, Inception_V3_Weights

weights = Inception_V3_Weights.DEFAULT
inception = inception_v3(weights=weights, aux_logits=True)  
inception.fc = nn.Identity()
inception.eval()


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [13]:
# ====================================
# STEP 5: FID CALCULATION FUNCTION
# ====================================
def calculate_fid(act1, act2):
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

    diff = mu1 - mu2
    covmean, _ = linalg.sqrtm(sigma1 @ sigma2, disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = diff @ diff + np.trace(sigma1 + sigma2 - 2 * covmean)
    return fid

In [14]:
# ====================================
# STEP 6: LOAD ACTIVATIONS
# ====================================
print("Extracting activations...")

act_real = get_activations(real_path, inception)
act_sd = get_activations(sd_path, inception)
act_mj = get_activations(mj_path, inception)
act_dalle = get_activations(dalle_path, inception)

Extracting activations...


Loading images from DALLEFaces: 100%|████████████████████████████████████████████████| 194/194 [00:04<00:00, 40.10it/s]


In [15]:
# ====================================
# STEP 7: CALCULATE FID SCORES
# ====================================
print("\nCalculating FID Scores...\n")

fid_sd = calculate_fid(act_real, act_sd)
fid_mj = calculate_fid(act_real, act_mj)
fid_dalle = calculate_fid(act_real, act_dalle)

print(f'FID (Stable Diffusion vs Real): {fid_sd:.2f}')
print(f'FID (Midjourney vs Real): {fid_mj:.2f}')
print(f'FID (DALL·E 2 vs Real): {fid_dalle:.2f}')


Calculating FID Scores...

FID (Stable Diffusion vs Real): 92.69
FID (Midjourney vs Real): 127.89
FID (DALL·E 2 vs Real): 119.72


Note: in this code we are NOT using the models, we are just using images generated by those models previously and comparing them with real images